# AWS-ASDI Linear Regression Example
Using the NOAA Global Historical Climatology Network Daily Dataset. This dataset contains over 200 years worth of climate data, and we will analyze the dataset through the use of data science tools that are scalable (modin and NumS). Because the tools we use are scalable, we are able to run the same code on a laptop and cluster of nodes. Before getting started, make sure you set `num_cpus` in `ray_init()` to the number of physical cores on the CPU running this notebook for optimal performance, otherwise Ray might automatically set it to logical cores that include hyperthreading.

Confirm everything is installed through:
```sh
pip3 install -r requirements
```

In [1]:
import ray
# ray.init(ignore_reinit_error=True, num_cpus=32, _temp_dir="/home/ubuntu/dlzou/aws-asdi/ray_temp");
ray.init(ignore_reinit_error=True)
import modin.pandas as pd
import pandas
from nums import numpy as nps
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import os

2021-07-25 17:31:50,806	INFO services.py:1274 -- View the Ray dashboard at http://127.0.0.1:8266


# Global Variables and DataFrames

In [2]:
# Global variables
elements = ["PRCP", "SNOW", "SNWD", "TMAX", "TMIN"]
years_range = list(range(1763, 2022))

In [3]:
stations = pd.read_fwf("s3://noaa-ghcn-pds/ghcnd-stations.txt", widths=[12, 9, 10, 7, 3, 31, 4, 4, 6], header=None)
stations.columns = ["ID", "LATITUDE", "LONGITUDE", "ELEVATION", "STATE", "NAME", "GSN FLAG", "HCN/CRN FLAG", "WMO ID"]
stations

,ID,LATITUDE,LONGITUDE,ELEVATION,STATE,NAME,GSN FLAG,HCN/CRN FLAG,WMO ID
0,ACW00011604,17.1167,-61.7833,10.1,NaN,ST JOHNS COOLIDGE FLD,NaN,NaN,NaN
1,ACW00011647,17.1333,-61.7833,19.2,NaN,ST JOHNS,NaN,NaN,NaN
2,AE000041196,25.3330,55.5170,34.0,NaN,SHARJAH INTER. AIRP,GSN,NaN,41196.0
3,AEM00041194,25.2550,55.3640,10.4,NaN,DUBAI INTL,NaN,NaN,41194.0
4,AEM00041217,24.4330,54.6510,26.8,NaN,ABU DHABI INTL,NaN,NaN,41217.0
...,...,...,...,...,...,...,...,...,...
118487,ZI000067969,-21.0500,29.3670,861.0,NaN,WEST NICHOLSON,NaN,NaN,67969.0
118488,ZI000067975,-20.0670,30.8670,1095.0,NaN,MASVINGO,NaN,NaN,67975.0
118489,ZI000067977,-21.0170,31.5830,430.0,NaN,BUFFALO RANGE,NaN,NaN,67977.0
118490,ZI000067983,-20.2000,32.6160,1132.0,NaN,CHIPINGE,GSN,NaN,67983.0


In [4]:
country_codes = pd.read_csv("s3://noaa-ghcn-pds/ghcnd-countries.txt", delimiter="\n", header=None)[0].str.extract('(?P<code>.{2})(?P<country>.{0,})')
country_codes

,code,country
0,AC,Antigua and Barbuda
1,AE,United Arab Emirates
2,AF,Afghanistan
3,AG,Algeria
4,AJ,Azerbaijan
...,...,...
214,WI,Western Sahara
215,WQ,Wake Island [United States]
216,WZ,Swaziland
217,ZA,Zambia


# Utils

In [5]:
def load_data(year, local=False):
    assert year in years_range
    if local:
        df = pd.read_csv(f"/home/ubuntu/ghcnd/csv/{year}.csv", header=None)
    else:
        df = pd.read_csv(f"s3://noaa-ghcn-pds/csv/{year}.csv", header=None)
    df.columns = ["ID", "YEAR/MONTH/DAY", "ELEMENT", "DATA VALUE", "M-FLAG", "Q-FLAG", "S-FLAG", "OBS-TIME"]
    df["YEAR/MONTH/DAY"] = pd.to_datetime(df["YEAR/MONTH/DAY"], format="%Y%m%d")
    return df

# Modeling

In [11]:
def linreg_matrix(years, features, target=None, convert_nps=False, local=False):
    assert isinstance(features, list)
    if target:
        assert target in features
    result = pd.DataFrame()
    
    for year in tqdm(years):
        df = load_data(year, local)
        df = df[df["ELEMENT"].isin(features)]
        df = pd.pivot_table(df, values=["DATA VALUE"], index=["ID", "YEAR/MONTH/DAY"], columns=["ELEMENT"])
        df = df.reset_index()
        df = df.merge(stations[["ID", "LATITUDE", "LONGITUDE", "ELEVATION"]], how="left", on="ID")
        if target:
            df = df.dropna(subset=[target])
        
        if result.empty:
            result = df
        else:
            result = result.append(df)
    
    if convert_nps:
        #return nums_modin.from_modin(result) # Uncomment this once bug gets fixed. 
        return nps.array(result.to_numpy().astype(np.double))
    return result

In [12]:
years = [1900, 1901]
features = ["PRCP", "SNOW", "TMAX", "TMIN"]
data = linreg_matrix(years, features, "SNOW")
data

  0%|          | 0/2 [00:00<?, ?it/s]

,ID,YEAR/MONTH/DAY,PRCP,SNOW,TMAX,TMIN,LATITUDE,LONGITUDE,ELEVATION
1064351,CA001010774,1900-01-01,0.0,0.0,NaN,NaN,48.5000,-123.3500,61.0
1064352,CA001010774,1900-01-02,74.0,0.0,NaN,NaN,48.5000,-123.3500,61.0
1064353,CA001010774,1900-01-03,10.0,0.0,NaN,NaN,48.5000,-123.3500,61.0
1064354,CA001010774,1900-01-04,76.0,0.0,NaN,NaN,48.5000,-123.3500,61.0
1064355,CA001010774,1900-01-05,84.0,0.0,NaN,NaN,48.5000,-123.3500,61.0
...,...,...,...,...,...,...,...,...,...
2884919,USW00094967,1901-12-27,10.0,10.0,-11.0,-78.0,46.9006,-95.0678,437.1
2884920,USW00094967,1901-12-28,28.0,28.0,0.0,-122.0,46.9006,-95.0678,437.1
2884921,USW00094967,1901-12-29,0.0,0.0,11.0,-78.0,46.9006,-95.0678,437.1
2884922,USW00094967,1901-12-30,0.0,0.0,44.0,-100.0,46.9006,-95.0678,437.1
